In [1]:
import pandas as pd
from Preprocess import * 
from FeatureExtraction import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
dataset_1 = pd.read_csv('data/FakeNewsNet.csv')
dataset_2 = pd.read_csv('data/news_articles.csv')


In [24]:
data = ["Testing four gram yes","Testing tri gram","two gram","gram"]
preprocessed_data=preprocess_text(data)
print(preprocessed_data)

for n,text in enumerate(preprocessed_data):
    for i in  range(1,5):
        if(len(text.split())<i):
            break
        print(i,'-gram:',n_gram(text,i))
        print(i,'Tf-Idf:',tf_idf(preprocessed_data,i)[n])

    print('')





['testing four gram yes', 'testing tri gram', 'kefak habibi lols', 'hello world']
1 -gram: ['testing', 'four', 'gram', 'yes']
1 Tf-Idf: [0.55528266 0.43779123 0.         0.         0.         0.
 0.43779123 0.         0.         0.55528266]
2 -gram: ['testing four', 'four gram', 'gram yes']
2 Tf-Idf: [0.57735027 0.57735027 0.         0.         0.         0.57735027
 0.         0.        ]
3 -gram: ['testing four gram', 'four gram yes']
3 Tf-Idf: [0.70710678 0.         0.70710678 0.        ]
4 -gram: ['testing four gram yes']
4 Tf-Idf: [1.]

1 -gram: ['testing', 'tri', 'gram']
1 Tf-Idf: [0.         0.52640543 0.         0.         0.         0.
 0.52640543 0.66767854 0.         0.        ]
2 -gram: ['testing tri', 'tri gram']
2 Tf-Idf: [0.         0.         0.         0.         0.         0.
 0.70710678 0.70710678]
3 -gram: ['testing tri gram']
3 Tf-Idf: [0. 0. 0. 1.]

1 -gram: ['kefak', 'habibi', 'lols']
1 Tf-Idf: [0.         0.         0.57735027 0.         0.57735027 0.57735027
 0